In [2]:
import numpy as np
import pandas as pd
import os
os.chdir('D:\pga16python')
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
df = pd.read_csv('train_2kmZucJ.csv')
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
df.shape

(7920, 3)

In [5]:
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [7]:
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [8]:
df1 = pd.DataFrame(df['tweet'])

In [9]:
df1['clean'] = df1[0:]

In [10]:
remove1 = lambda x: re.sub("(@[a-zA-Z0-9]+)|(@[0-9A-Za-z  \t]+)|(\w+:\/\/\S+)"," ",x)

In [11]:
df1['clean'] = df1['clean'].map(remove1)

In [12]:
def remove(txt,pattern):
    review = re.findall(pattern,txt)
    for word in review:
        txt = re.sub(word,'',txt)
    return txt

In [13]:
df1['clean'] = np.vectorize(remove)(df1['clean'],'#')

In [14]:
#remove2 = lambda x: re.findall(r"[^?!,.^:)(]+",x)

In [15]:
#df1['clean'] = df1['clean'].map(remove2)

In [16]:
import string

In [17]:
df1['clean'] = df1['clean'].apply(lambda x: ''.join([p for p in x if p not in string.punctuation]))

In [18]:
stemmer = PorterStemmer()
wordnet = WordNetLemmatizer()

In [19]:
corpus =[]
for i in range(0,7920):
    review = re.sub('[^a-zA-Z]',' ',df1.loc[i,'clean'])
    review = review.lower()
    review = review.split()
    review = [wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english')) ]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer()

In [21]:
X1 = tf_idf.fit_transform(corpus).toarray()

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import  MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix

In [23]:
y = df['label']
X_train,X_test,y_train,y_test = train_test_split(X1,y,test_size = 0.3,random_state = 0)

In [25]:
clf = MultinomialNB()
clf.fit(X_train,y_train)
accuracy_score(y_train,clf.predict(X_train))

0.9085497835497836

In [26]:
accuracy_score(y_test,clf.predict(X_test))

0.843013468013468